In [1]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

parser = argparse.ArgumentParser()
parser.add_argument('--image_folder', type=str, default='data/samples', help='path to dataset')
parser.add_argument('--config_path', type=str, default='config/yolov3.cfg', help='path to model config file')
parser.add_argument('--weights_path', type=str, default='weights/yolov3.weights', help='path to weights file')
parser.add_argument('--class_path', type=str, default='data/coco.names', help='path to class label file')
parser.add_argument('--conf_thres', type=float, default=0.8, help='object confidence threshold')
parser.add_argument('--nms_thres', type=float, default=0.4, help='iou thresshold for non-maximum suppression')
parser.add_argument('--batch_size', type=int, default=1, help='size of the batches')
parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
parser.add_argument('--img_size', type=int, default=416, help='size of each image dimension')
parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use cuda if available')
opt = parser.parse_args(args=['--config_path', '/home/dllab/final/yolov3.cfg', '--class_path', '/home/dllab/final/clothes.names', '--image_folder', '/home/dllab/clothes/clothes_test/images', '--weights_path', '/tmp/work/105.pt', '--img_size', '640'])
print(opt)

cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs('output', exist_ok=True)

# Set up model
model = Darknet(opt.config_path, img_size=opt.img_size)
#model.load_weights(opt.weights_path)
model = torch.load(opt.weights_path)        #use this to load

if cuda:
    model.cuda()

model.eval() # Set in evaluation mode

dataloader = DataLoader(ImageFolder(opt.image_folder, img_size=opt.img_size),
                        batch_size=opt.batch_size, shuffle=False, num_workers=opt.n_cpu)

classes = load_classes(opt.class_path) # Extracts class labels from file

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

imgs = []           # Stores image paths
img_detections = [] # Stores detections for each image index

print ('\nPerforming object detection:')
prev_time = time.time()
for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
    # Configure input
    input_imgs = Variable(input_imgs.type(Tensor))
    
    input_imgs = input_imgs.to('cuda')

    # Get detections
    with torch.no_grad():
        detections = model(input_imgs)
        detections = non_max_suppression(detections, 80, opt.conf_thres, opt.nms_thres)


    # Log progress
    current_time = time.time()
    inference_time = datetime.timedelta(seconds=current_time - prev_time)
    prev_time = current_time
    print ('\t+ Batch %d, Inference Time: %s' % (batch_i, inference_time))

    # Save image and detections
    imgs.extend(img_paths)
    img_detections.extend(detections)

# Bounding-box colors
cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

print ('\nSaving images:')
# Iterate through images and save plot of detections
for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):

    print ("(%d) Image: '%s'" % (img_i, path))

    # Create plot
    img = np.array(Image.open(path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # The amount of padding that was added
    pad_x = max(img.shape[0] - img.shape[1], 0) * (opt.img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (opt.img_size / max(img.shape))
    # Image height and width after padding is removed
    unpad_h = opt.img_size - pad_y
    unpad_w = opt.img_size - pad_x

    # Draw bounding boxes and labels of detections
    if detections is not None:
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        bbox_colors = random.sample(colors, n_cls_preds)
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

            print ('\t+ Label: %s, Conf: %.5f' % (classes[int(cls_pred)], cls_conf.item()))

            # Rescale coordinates to original dimensions
            box_h = ((y2 - y1) / unpad_h) * img.shape[0]
            box_w = ((x2 - x1) / unpad_w) * img.shape[1]
            y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
            x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]

            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            # Create a Rectangle patch
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2,
                                    edgecolor=color,
                                    facecolor='none')
            # Add the bbox to the plot
            ax.add_patch(bbox)
            # Add label
            plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top',
                    bbox={'color': color, 'pad': 0})

    # Save generated image with detections
    plt.axis('off')
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.savefig('output/%d.png' % (img_i), bbox_inches='tight', pad_inches=0.0)
    plt.close()


Namespace(batch_size=1, class_path='/home/dllab/final/clothes.names', conf_thres=0.8, config_path='/home/dllab/final/yolov3.cfg', image_folder='/home/dllab/clothes/clothes_test/images', img_size=640, n_cpu=8, nms_thres=0.4, use_cuda=True, weights_path='/tmp/work/105.pt')


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))



Performing object detection:


/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  

	+ Batch 0, Inference Time: 0:00:01.694890
	+ Batch 1, Inference Time: 0:00:00.070207
	+ Batch 2, Inference Time: 0:00:00.077133
	+ Batch 3, Inference Time: 0:00:00.118288
	+ Batch 4, Inference Time: 0:00:00.038803
	+ Batch 5, Inference Time: 0:00:00.056311
	+ Batch 6, Inference Time: 0:00:00.041771
	+ Batch 7, Inference Time: 0:00:00.073316
	+ Batch 8, Inference Time: 0:00:00.085551
	+ Batch 9, Inference Time: 0:00:00.103536
	+ Batch 10, Inference Time: 0:00:00.083042
	+ Batch 11, Inference Time: 0:00:00.029000
	+ Batch 12, Inference Time: 0:00:00.077882
	+ Batch 13, Inference Time: 0:00:00.039434
	+ Batch 14, Inference Time: 0:00:00.081344
	+ Batch 15, Inference Time: 0:00:00.081587
	+ Batch 16, Inference Time: 0:00:00.036329
	+ Batch 17, Inference Time: 0:00:00.078628
	+ Batch 18, Inference Time: 0:00:00.094967
	+ Batch 19, Inference Time: 0:00:00.080653
	+ Batch 20, Inference Time: 0:00:00.038072
	+ Batch 21, Inference Time: 0:00:00.093242
	+ Batch 22, Inference Time: 0:00:00.07684

...

/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


(20) Image: '/home/dllab/clothes/clothes_test/images/2021.jpg'
	+ Label: Deep+LT, Conf: 1.00000
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Deep+LU, Conf: 0.99995
	+ Label: Light+LT, Conf: 0.99415
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Light+ST, Conf: 0.96849
	+ Label: Light+ST, Conf: 0.99393
	+ Label: Light+ST, Conf: 0.99520
(21) Image: '/home/dllab/clothes/clothes_test/images/2022.jpg'
	+ Label: Light+LU, Conf: 0.98963
	+ Label: Deep+SU, Conf: 0.99582
	+ Label: Deep+SU, Conf: 0.99996
	+ Label: Light+ST, Conf: 0.97224
	+ Label: Light+ST, Conf: 0.96368
	+ Label: Light+LT, Conf: 0.96497
	+ Label: Light+LT, Conf: 0.99981
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Deep+LU, Conf: 0.99988
	+ Label: Deep+LT, Conf: 0.93463
	+ Label: Deep+LT, Conf: 0.42427
	+ Label: Deep+ST, Conf: 0.76528
	+ Label: Deep+ST, Conf: 0.99986
	+ Label: Deep+ST, Conf: 0.17031
(22) Image: '/home/dllab/clothes/clothes_test/images/2023.jpg'
	+ Label: 

(39) Image: '/home/dllab/clothes/clothes_test/images/2040.jpg'
	+ Label: Light+LT, Conf: 0.99963
	+ Label: Deep+LU, Conf: 0.99983
	+ Label: Light+ST, Conf: 0.99739
	+ Label: Light+ST, Conf: 0.95242
	+ Label: Light+ST, Conf: 0.98976
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 0.99997
	+ Label: Deep+ST, Conf: 0.84804
	+ Label: Deep+ST, Conf: 0.99201
	+ Label: Deep+ST, Conf: 0.99971
	+ Label: Deep+ST, Conf: 0.99984
	+ Label: Deep+ST, Conf: 0.99916
	+ Label: Deep+ST, Conf: 0.99438
	+ Label: Deep+LT, Conf: 0.99780
(40) Image: '/home/dllab/clothes/clothes_test/images/2041.jpg'
	+ Label: Light+ST, Conf: 0.99030
	+ Label: Deep+LU, Conf: 0.99876
	+ Label: Deep+SU, Conf: 0.99366
	+ Label: Deep+LT, Conf: 0.99946
	+ Label: Deep+LT, Conf: 0.99956
	+ Label: Deep+LT, Conf: 0.99947
	+ Label: Deep+ST, Conf: 0.99980
	+ Label: Deep+ST, Conf: 0.99900
	+ Label: Deep+ST, Conf: 0.99980
(41) Image: '/home/dllab/clothes/clothes_test/images/2042.jpg'
	+ Label: Deep+SU, Conf: 0.93811
	+ Label: Ligh

(62) Image: '/home/dllab/clothes/clothes_test/images/2063.jpg'
	+ Label: Deep+LT, Conf: 1.00000
	+ Label: Deep+LT, Conf: 0.99300
	+ Label: Deep+SU, Conf: 0.95260
	+ Label: Deep+LU, Conf: 0.98648
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 1.00000
(63) Image: '/home/dllab/clothes/clothes_test/images/2064.jpg'
	+ Label: Deep+ST, Conf: 0.92849
	+ Label: Deep+ST, Conf: 0.71867
	+ Label: Deep+LT, Conf: 0.92859
	+ Label: Deep+LU, Conf: 0.99853
	+ Label: Deep+LU, Conf: 1.00000
	+ Label: Light+ST, Conf: 0.25329
	+ Label: Light+ST, Conf: 0.34933
	+ Label: Light+ST, Conf: 0.99906
	+ Label: Light+SU, Conf: 0.54522
(64) Image: '/home/dllab/clothes/clothes_test/images/2065.jpg'
	+ Label: Light+ST, Conf: 0.99999
	+ Label: Light+ST, Conf: 0.99916
	+ Label: Deep+ST, Conf: 0.95902
	+ Label: Deep+LU, Conf: 0.99999
	+ Label: Light+LT, Conf: 0.88673
	+ Label: Light+LT, Conf: 1.00000
	+ Label: Light+LT, Conf: 0.70628
(65) Image: '/home/dllab/clothes/clothes_te

(85) Image: '/home/dllab/clothes/clothes_test/images/2086.jpg'
	+ Label: Deep+SU, Conf: 0.99891
	+ Label: Deep+LU, Conf: 0.99998
	+ Label: Light+ST, Conf: 1.00000
(86) Image: '/home/dllab/clothes/clothes_test/images/2087.jpg'
	+ Label: Light+LT, Conf: 0.92216
	+ Label: Light+ST, Conf: 0.99997
	+ Label: Light+ST, Conf: 0.96829
	+ Label: Deep+ST, Conf: 0.99366
	+ Label: Deep+ST, Conf: 0.99999
	+ Label: Deep+ST, Conf: 0.99999
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 0.99995
	+ Label: Deep+ST, Conf: 0.99996
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+ST, Conf: 0.99961
	+ Label: Deep+ST, Conf: 0.71528
	+ Label: Deep+ST, Conf: 0.83404
	+ Label: Deep+LU, Conf: 0.98445
	+ Label: Deep+LT, Conf: 0.87133
	+ Label: Deep+LT, Conf: 0.93991
(87) Image: '/home/dllab/clothes/clothes_test/images/2088.jpg'
	+ Label: Deep+ST, Conf: 1.00000
	+ Label: Deep+LT, Conf: 0.99878
(88) Image: '/home/dllab/clothes/clothes_test/images/2089.jpg'
	+ Label: Light+ST, Conf: 0.58867
	+ Label: Light+S